**SPEAKER DIARIZATION, SENTIMENT ANALYSIS AND SUMMARIZATION**

Libraries to be installed

In [1]:
!pip install --q git+https://github.com/m-bain/whisperx.git
!pip install transformers
!pip install bert
!pip install bert-extractive-summarizer
!pip install nltk

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.4 MB/s eta 0:00:00
 

**SPEAKER DIARIZATION**

In [3]:
device = 'cuda'
batch_size =16
compute_type = 'float16'

In [4]:
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving why.wav to why.wav


In [5]:
audio_file = 'why.wav'

In [6]:
model = whisperx.load_model("large-v2",device,compute_type = compute_type)

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 11.4MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [7]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio,batch_size=batch_size)
print(result['segments'])

Detected language: en (0.99) in first 30s of audio...
[{'text': " Why is it, when something happens, it is always you three? Believe me, Professor, I've been asking myself the same question for six years.", 'start': 1.049, 'end': 11.698}]


In [8]:
model_a,metadata = whisperx.load_align_model(language_code=result['language'],device=device)

result = whisperx.align(result['segments'],model_a,
                        metadata,
                        audio,
                        device,
                        return_char_alignments = False)
print(result['segments'])

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 206MB/s]


[{'start': 1.089, 'end': 5.942, 'text': ' Why is it, when something happens, it is always you three?', 'words': [{'word': 'Why', 'start': 1.089, 'end': 1.33, 'score': 0.876}, {'word': 'is', 'start': 1.47, 'end': 1.59, 'score': 0.877}, {'word': 'it,', 'start': 1.671, 'end': 1.751, 'score': 0.851}, {'word': 'when', 'start': 2.132, 'end': 2.332, 'score': 0.708}, {'word': 'something', 'start': 2.393, 'end': 2.834, 'score': 0.936}, {'word': 'happens,', 'start': 2.894, 'end': 3.415, 'score': 0.835}, {'word': 'it', 'start': 3.636, 'end': 3.716, 'score': 0.98}, {'word': 'is', 'start': 3.817, 'end': 3.917, 'score': 0.666}, {'word': 'always', 'start': 4.358, 'end': 4.92, 'score': 0.748}, {'word': 'you', 'start': 5.06, 'end': 5.341, 'score': 0.842}, {'word': 'three?', 'start': 5.601, 'end': 5.942, 'score': 0.831}]}, {'start': 5.962, 'end': 11.076, 'text': "Believe me, Professor, I've been asking myself the same question for six years.", 'words': [{'word': 'Believe', 'start': 5.962, 'end': 7.948, 

In [9]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token ="your hugging face token here" ,device=device)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [10]:
diarize = diarize_model(audio, min_speakers = 2, max_speakers = 2)

In [11]:
diarize

,segment,label,speaker,start,end
0,[ 00:00:01.027 --> 00:00:06.035],A,SPEAKER_01,1.027165,6.035654
1,[ 00:00:07.614 --> 00:00:11.196],B,SPEAKER_00,7.614601,11.196944


In [12]:
diarize.speaker.unique()

array(['SPEAKER_01', 'SPEAKER_00'], dtype=object)

In [13]:
result = whisperx.assign_word_speakers(diarize,result)
print(diarize)
print(result['segments'])

                             segment label     speaker     start        end  \
0  [ 00:00:01.027 -->  00:00:06.035]     A  SPEAKER_01  1.027165   6.035654   
1  [ 00:00:07.614 -->  00:00:11.196]     B  SPEAKER_00  7.614601  11.196944   

   intersection      union  
0     -4.780346  10.048835  
1      0.260000   3.582343  
[{'start': 1.089, 'end': 5.942, 'text': ' Why is it, when something happens, it is always you three?', 'words': [{'word': 'Why', 'start': 1.089, 'end': 1.33, 'score': 0.876, 'speaker': 'SPEAKER_01'}, {'word': 'is', 'start': 1.47, 'end': 1.59, 'score': 0.877, 'speaker': 'SPEAKER_01'}, {'word': 'it,', 'start': 1.671, 'end': 1.751, 'score': 0.851, 'speaker': 'SPEAKER_01'}, {'word': 'when', 'start': 2.132, 'end': 2.332, 'score': 0.708, 'speaker': 'SPEAKER_01'}, {'word': 'something', 'start': 2.393, 'end': 2.834, 'score': 0.936, 'speaker': 'SPEAKER_01'}, {'word': 'happens,', 'start': 2.894, 'end': 3.415, 'score': 0.835, 'speaker': 'SPEAKER_01'}, {'word': 'it', 'start': 3.

In [14]:
import json

def convert_to_desired_format(data):
    result = {}
    for entry in data:
        speaker = entry['speaker']
        text = entry['text'].strip()
        if speaker not in result:
            result[speaker] = []
        result[speaker].append(text)
    return result

converted_data = convert_to_desired_format(result['segments'])

for speaker, texts in converted_data.items():
    for text in texts:
        print(f"{speaker}:\"{text}\"")


SPEAKER_01:"Why is it, when something happens, it is always you three?"
SPEAKER_00:"Believe me, Professor, I've been asking myself the same question for six years."


**SUMMARIZATION**

In [16]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
summarizer = Summarizer()



config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [18]:
def dict_to_string(input_dict):

    string_parts = []

    for key, value_list in input_dict.items():
        joined_value = ', '.join(value_list)
        formatted_pair = f"'{key}': '{joined_value}'"
        string_parts.append(formatted_pair)

    final_string = ', '.join(string_parts)

    return final_string

output_string = dict_to_string(converted_data)




In [19]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary = summarizer(output_string, max_length=50, min_length=25, do_sample=False)

print(summary[0]['summary_text'])


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

'Why is it, when something happens, it is always you three?', 'SPEAKER_00': 'Believe me, Professor, I've been asking myself the same question for six years'


In [20]:
from transformers import pipeline


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

conversation = converted_data

sentences = [f"{key}: {value}" for key, value in conversation.items()]

text = " ".join(sentences)

summary = summarizer(text, max_length=50, min_length=25, do_sample=False)

summarized_text = summary[0]['summary_text']

print(summarized_text)

"Why is it, when something happens, it is always you three?" "Believe me, Professor, I've been asking myself the same question for six years"


**SENTIMENT ANALYSIS**

In [21]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

text = summarized_text

sentiment_scores = sia.polarity_scores(text)

if sentiment_scores['compound'] >= 0.05:
    sentiment = 'Positive'
elif sentiment_scores['compound'] <= -0.05:
    sentiment = 'Negative'
else:
    sentiment = 'Neutral'

print(f"Sentiment: {sentiment}")
print(f"Sentiment Scores: {sentiment_scores}")

Sentiment: Neutral
Sentiment Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
